In [8]:
import pandas as pd
import plotly.express as px
from sklearn.feature_extraction.text import CountVectorizer
from nltk.util import ngrams
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
import nltk

nltk.download('punkt')
nltk.download('stopwords')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\srinivas\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\srinivas\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [21]:
df=pd.read_csv('main.csv')

In [22]:
df.head()

,Unnamed: 0,DATE,AUTHOR,TITLE
0,0,2017-05-31,\nTasha Robinson\n,Agent Cooper in Twin Peaks is the audience: on...
1,1,2017-05-30,\nSam Byford\n,"AI, the humanity!"
2,2,2017-05-25,\nKaitlyn Tiffany\n,The Viral Machine
3,3,2017-05-22,\nNick Statt\n,How Anker is beating Apple and Samsung at thei...
4,4,2017-05-15,\nKwame Opam\n,Tour Black Panther’s reimagined homeland with ...


In [23]:
len(df)

2019612

In [24]:
df['DATE'] = df['DATE'].str.extract(r'(\d{4}-\d{2}-\d{2})')


Inference from Graphs:
Number of Articles Published Over Time:
This graph displays a significant rise in article publications starting around 2015, peaking between 2016 and 2019, and then showing a sharp drop. This trend might indicate a period of high activity or interest in the covered topics, potentially due to political cycles, technological advancements, or organizational focus shifts.


In [25]:
df['DATE'] = pd.to_datetime(df['DATE'])

# Set the 'DATE' column as the index (optional since Plotly can handle date columns directly)
df.set_index('DATE', inplace=True)

# Resample the data by day and count the number of articles
daily_articles = df.resample('D').count()

# Reset index to use DATE as a column for Plotly
daily_articles = daily_articles.reset_index()

# Using Plotly to create an interactive time series plot
fig = px.line(daily_articles, x='DATE', y='TITLE',
              labels={'DATE': 'Date', 'TITLE': 'Number of Articles'},
              title='Number of Articles Published Over Time')

# Show the plot
fig.show()

Top 5 Authors by Number of Articles:
WIRED Staff leads by a substantial margin in the number of articles published, followed by the Associated Press, Field Level Media, John Bowden, and The Associated Press again. This data points to WIRED Staff's dominant contribution to the dataset, suggesting that their content significantly influences the themes and styles represented.

In [26]:
author_counts = df['AUTHOR'].value_counts()

# Filter to get the top 5 authors
top_authors = author_counts.head(5)

# Creating the bar graph using Plotly
fig = px.bar(top_authors, 
             x=top_authors.index, 
             y=top_authors.values,
             title='Top 5 Authors by Number of Articles',
             labels={'x': 'Author', 'y': 'Number of Articles'},
             text=top_authors.values)

# Add customization: show the exact count on top of each bar
fig.update_traces(texttemplate='%{text}', textposition='outside')

# Show the plot
fig.show()

In [33]:
author_counts = df['AUTHOR'].value_counts()

# Filter to get the top 5 authors
top_authors = author_counts.head(2000)

# Creating the bar graph using Plotly
# fig = px.bar(top_authors, 
#              x=top_authors.index, 
#              y=top_authors.values,
#              title='Top 5 Authors by Number of Articles',
#              labels={'x': 'Author', 'y': 'Number of Articles'},
#              text=top_authors.values)

# # Add customization: show the exact count on top of each bar
# fig.update_traces(texttemplate='%{text}', textposition='outside')

# # Show the plot
# fig.show()
top_authors.tail()

AUTHOR
Tom Finn             187
Ludwig Burger        187
Alicia Cohn          187
Lisa W. Foderaro     187
J. Weston Phippen    187
Name: count, dtype: int64

In [34]:
# Assuming author_counts is calculated as df['AUTHOR'].value_counts()
top_authors = author_counts.head(2000)

In [35]:
# Assuming 'df' is your original DataFrame
top_authors_df = df[df['AUTHOR'].isin(top_authors.index)]


In [36]:
# Group by 'AUTHOR' and apply the head() function to limit to 200 entries per author
downsampled_df = top_authors_df.groupby('AUTHOR').head(200)


In [38]:
downsampled_df.head()

,Unnamed: 0,AUTHOR,TITLE
DATE,,,
2017-06-06,137,Adi Robertson,Breaking down Apple’s new augmented reality pl...
2017-06-06,138,Russell Brandom,Apple's new anti-tracking system will make Goo...
2017-06-06,139,Rachel Becker,Wonder Woman's chemical weapon makes no scient...
2017-06-05,140,Russell Brandom,Feds look to experimental light-field cameras ...
2017-06-01,141,Angela Chen,Scientists remotely move a mouse’s whiskers wi...


In [39]:
print(f"Unique authors: {len(downsampled_df['AUTHOR'].unique())}")
print(f"Total records: {len(downsampled_df)}")

Unique authors: 2000
Total records: 399158


In [40]:
downsampled_df.to_csv('DOWNSAMPLED.csv')

In [11]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import classification_report


In [41]:
from sklearn.feature_extraction.text import TfidfVectorizer

# Assuming 'downsampled_df' is your DataFrame with 'TITLE' and 'AUTHOR'
vectorizer = TfidfVectorizer(stop_words='english', max_features=1000)
X = vectorizer.fit_transform(downsampled_df['TITLE'])
y = downsampled_df['AUTHOR']


In [42]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


In [43]:
from sklearn.naive_bayes import MultinomialNB

model = MultinomialNB()


In [44]:
model.fit(X_train, y_train)


MultinomialNB()

In [45]:
from sklearn.metrics import classification_report

y_pred = model.predict(X_test)
print(classification_report(y_test, y_pred))


c:\Users\srinivas\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1509: UndefinedMetricWarning:

Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.

c:\Users\srinivas\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1509: UndefinedMetricWarning:

Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.



                                                                            precision    recall  f1-score   support

                                                              AJ Dellinger       0.00      0.00      0.00        46
                                                             AJ Willingham       0.00      0.00      0.00        46
                                                               AWR Hawkins       0.07      0.31      0.11        35
                                                                Aaron Bady       0.00      0.00      0.00        32
                                                              Aaron Gordon       0.17      0.03      0.04        39
                                                              Aaron Holmes       0.00      0.00      0.00        44
                                                               Aaron Klein       0.17      0.18      0.18        38
                                                               Aaron Ru

c:\Users\srinivas\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1509: UndefinedMetricWarning:

Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.



In [47]:
from sklearn.metrics import accuracy_score

accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy: {accuracy * 100:.2f}%")


Accuracy: 12.23%


In [50]:
!python -m spacy download en_core_web_sm


     ---------------------------------------- 0.0/12.8 MB ? eta -:--:--
     --------------------------------------- 0.0/12.8 MB 660.6 kB/s eta 0:00:20
      --------------------------------------- 0.2/12.8 MB 2.1 MB/s eta 0:00:07
     - -------------------------------------- 0.3/12.8 MB 2.6 MB/s eta 0:00:05
     -- ------------------------------------- 0.7/12.8 MB 4.1 MB/s eta 0:00:03
     --- ------------------------------------ 1.2/12.8 MB 5.6 MB/s eta 0:00:03
     ----- ---------------------------------- 1.7/12.8 MB 6.3 MB/s eta 0:00:02
     ------ --------------------------------- 2.2/12.8 MB 7.0 MB/s eta 0:00:02
     -------- ------------------------------- 2.8/12.8 MB 7.7 MB/s eta 0:00:02
     ---------- ----------------------------- 3.3/12.8 MB 8.2 MB/s eta 0:00:02
     ----------- ---------------------------- 3.8/12.8 MB 8.4 MB/s eta 0:00:02
     ------------- -------------------------- 4.4/12.8 MB 8.8 MB/s eta 0:00:01
     --------------- ------------------------ 5.0/12.8 MB 

In [15]:
downsampled_df=pd.read_csv('DOWNSAMPLED.csv')

In [16]:
import spacy
nlp = spacy.load('en_core_web_sm', disable=['parser', 'ner'])

def lemmatize_texts(docs):
    # Process texts as a batch and return them
    return [" ".join([token.lemma_ for token in doc if not token.is_stop]) for doc in nlp.pipe(docs)]

# Applying the function to batches of the data
downsampled_df['cleaned'] = lemmatize_texts(downsampled_df['TITLE'])


In [9]:
# downsampled_df=pd.read_csv('CLEANED_DOWN.csv')

In [17]:
from sklearn.feature_extraction.text import ENGLISH_STOP_WORDS

custom_stop_words = list(ENGLISH_STOP_WORDS) + ['additional', 'stopwords']
vectorizer = TfidfVectorizer(stop_words=custom_stop_words)


In [18]:
vectorizer = TfidfVectorizer(ngram_range=(1, 3))
X = vectorizer.fit_transform(downsampled_df['cleaned'])


In [24]:
vectorizer = TfidfVectorizer(min_df=3, max_df=0.7, ngram_range=(1, 3))
X = vectorizer.fit_transform(downsampled_df['cleaned'])


In [25]:
y = downsampled_df['AUTHOR']


In [26]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)


In [27]:
model = MultinomialNB()


In [28]:
model.fit(X_train, y_train)

MultinomialNB()

In [30]:
from sklearn.metrics import classification_report

y_pred = model.predict(X_test)
print(classification_report(y_test, y_pred))


c:\Users\srinivas\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\srinivas\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


                                                                            precision    recall  f1-score   support

                                                              AJ Dellinger       0.00      0.00      0.00        61
                                                             AJ Willingham       0.86      0.10      0.18        61
                                                               AWR Hawkins       0.12      0.62      0.20        50
                                                                Aaron Bady       0.51      0.50      0.50        56
                                                              Aaron Gordon       0.39      0.11      0.17        65
                                                              Aaron Holmes       0.33      0.05      0.08        65
                                                               Aaron Klein       0.48      0.42      0.45        57
                                                               Aaron Ru

c:\Users\srinivas\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


In [31]:
from sklearn.metrics import accuracy_score

accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy: {accuracy * 100:.2f}%")


Accuracy: 24.37%


In [32]:
from sklearn.svm import SVC

model1 = SVC(kernel='rbf', C=1.0, random_state=42)  # 'C' is the regularization parameter

# Train the model
model1.fit(X_train, y_train)

SVC(random_state=42)

In [33]:
from sklearn.metrics import classification_report

y_pred = model1.predict(X_test)
print(classification_report(y_test, y_pred))

Summary of the Data Wrangling File (DW.ipynb):
In the Data Wrangling file, we processed and analyzed various aspects of a large dataset of articles. Key tasks included cleaning data, extracting and visualizing key metrics like publication trends and author contributions, and applying statistical models to uncover patterns in the data such as sentiment analysis and topic modeling. The insights drawn from these visualizations and analyses help in understanding the distribution and characteristics of the content, such as author productivity, the temporal dynamics of publications, and the sentiment embedded in the articles. This comprehensive approach aids in making informed decisions or hypotheses about the data's underlying trends and themes.